# To be loaded before in memory

In [1]:
import torch
from PIL import Image
import open_clip
import pandas as pd
import time
# from google.colab import drive
# drive.mount('/content/gdrive')

device = torch.device("cuda")

model, _, preprocess = open_clip.create_model_and_transforms('ViT-H-14', pretrained='laion2b_s32b_b79k', precision="fp16", device=device)

C:\Users\Pana Demopoulos\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
textcat=torch.load("C:/Users/Pana Demopoulos/AppData/Local/Programs/Python/Python310/textcat0-377602.pt").to(device)

In [3]:
df = pd.read_csv('C:/Users/Pana Demopoulos/AppData/Local/Programs/Python/Python310/our_foods.csv', header=None, dtype = str)
df2 = df.iloc[:,1]
df1 = df.iloc[:,0]
print(df2[0])
dflist=df2.values.tolist()
dflist_str = [str(x) for x in dflist]

start=time.time()
textcat /= textcat.norm(dim=-1, keepdim=True)
end = time.time()
print((end-start),"sec")

Hummus, commercial
0.12499761581420898 sec


# Real time inference code

In [5]:
image = preprocess(Image.open("C:/Users/Pana Demopoulos/AppData/Local/Programs/Python/Python310/junlin1.jpg")).unsqueeze(0).to(device)

def clippify(image):
  with torch.no_grad(), torch.cuda.amp.autocast():
    start=time.time()
    image_features = model.encode_image(image)
  ##    text_features = model.encode_text(text)
    end = time.time()
    print((end-start),"sec")
    image_features /= image_features.norm(dim=-1, keepdim=True)
    start=time.time()
    text_probs = (100.0 * image_features @ textcat.T).softmax(dim=-1)
    end = time.time()
    print((end-start),"sec")

  # print(image_features, image_features.size())
  # print(textcat[100000:100100], textcat.size())
  # print(text_probs, text_probs.size())
  # print(text_probs[100000:100100])
  probs_list  = text_probs[0].tolist()
  # print(len(probs_list))
  # print(probs_list[0])

  biglist=[biglist[0] for biglist in sorted(enumerate(probs_list), key=lambda i:i[1], reverse=True)]
  top_i_list=biglist[:10]
  # print(top_i_list)
  top10foods=[]
  for x in top_i_list:
    top10foods.append([(df.iloc[x,:]).tolist(), probs_list[x]])
  print("Label probs:\n", top10foods)  # prints: [[1., 0., 0.]]
  return top10foods

output=clippify(image)

0.03099799156188965 sec
0.0 sec
Label probs:
 [[['861536', 'WHITE MEAT CHICKEN FRITTER IN A SWEET & SOUR GLAZE WITH VEGETABLES OVER RICE, SWEET & SOUR CHICKEN', '397.0', 'g'], 0.15592412650585175], [['679134', 'CHICKEN BREAST FRITTER IN A SWEET & SOUR GLAZE WITH VEGETABLES OVER RICE, SWEET & SOUR CHICKEN', '397.0', 'g'], 0.07840356975793839], [['986831', 'SWEET & SOUR CHICKEN BREADED CHICKEN WHITE MEAT AND VEGETABLES IN SWEET AND SOUR SAUCE WITH FRIED RICE, SWEET & SOUR CHICKEN', '312.0', 'g'], 0.06919091194868088], [['997049', 'WHITE MEAT CHICKEN, VEGETABLES & FRIED RICE WITH OYSTER FLAVORED SAUCE, CHICKEN FRIED RICE', '312.0', 'g'], 0.06919091194868088], [['649568', 'TENDER WHITE MEAT CHICKEN IN A TANGY SWEET & SOUR SAUCE WITH RICE, SWEET & SOUR CHICKEN', '283.0', 'g'], 0.04609108343720436], [['531861', 'SWEET & SOUR CHICKEN BREADED CHICKEN WHITE MEAT AND VEGETABLES IN SWEET AND SOUR SAUCE WITH FRIED RICE', '312.0', 'g'], 0.03479137644171715], [['643408', 'CHICKEN FLAVORED FRIED RICE